# 5 - Merge Datasets

Merging datasets is a very common practice to enrich or validate the values we have. It's easy to do but it's better used when done practically.


In *4 - Cleaning Real Data* we used data from [hoopshype.com](hoopshype.com) that included Actual Salaries and Adjusted Salaries. In this one, we're going to create our own Adjusted Salaries using the dataset from *Appendix B - Inflation Rate Dataset*

In [ ]:
import datetime
import pandas as pd
import pathlib
import utils

In [ ]:
BASE_DIR = pathlib.Path().resolve()
COURSES_DIR = BASE_DIR / 'course'
APPENDIX_DIR = BASE_DIR / 'appendix'
DATASET_PATH = BASE_DIR / 'datasets'
SAMPLES_DIR = BASE_DIR / 'samples'
INPUT_PATH = SAMPLES_DIR / '4-player-salaries-cleaned.csv'
INFLATION_DATA_PATH = DATASET_PATH / 'inflation-rate.csv'
print(f'Dataset *{INPUT_PATH.name}* exists:', INPUT_PATH.exists())

In [ ]:
# Import Dataset from 4 - Cleaning Real Data
df = pd.read_csv(INPUT_PATH)

In [ ]:
df.head()

Going forward, we are going to be doing a lot of analysis in 2020 dollars (2020 has the most up to date data as of October 2021).

We're going to assume a few things about this scraped data:
- Player names are correct (`player` column)
- Salary (`salary` column) listed is their actual salary
- Start Year is accurate (`year_start` column)

Given these assumptions, we're going to create our own Adjust Salary column to illustrate how to merge data.

In [ ]:
inflation_df = pd.read_csv(INFLATION_DATA_PATH)

In [ ]:
inflation_df.head()

*Appendix B - Inflation Rate Dataset* shows exactly where and how the dataset for `inflation_df` is created. 

In [ ]:
inflation_df.set_index('date', inplace=True)

Typically, the DataFrame index is auto-incrementing integers (0, 1, 2, 3, 4, ...) but it can be a time series index (ie based in dates).

Setting our index to a date-like string (ie `YYYY-MM-DD`) will result in time series data.

The nice thing about this is we can take a slice this data in a cool way:

In [ ]:
year_start = 2000
year_end = 2005
inflation_df[f"{year_start}": f"{year_end}"]

Now we see a subset of our dataset. You can treat this as a new dataframe if you need or we can use it when enriching our data. We're not going to use this type of slicing in this guide but it is nice to see it in action.

In [ ]:
year_start = 2000
year_end = 2001
inflation_df[f"{year_start}": f"{year_end}"]

This slice should help show us something interesting: for the `year_start` and `year_end` we choose, it has 2 new values that are related both the `inflation_rate_percent` and `multiplier`.

Now we *can* use an apply here to enrich our data:
```python
def merge_data_via_lookup(row):
    year_start = row['year_start']
    year_end = row['year_end']
    new_data = inflation_df[f"{year_start}": f"{year_end}"]
    row['multiplier'] = new_data['multiplier'].values[0]
    return row
    
df.apply(merge_data_via_lookup, axis=1)
```

Technically speaking, this would work but it's not efficient and it can lead to confusion. Let's use the built-in `merge` function instead.

Since `year_start` from `df` and the index (ie the `date` column) on `inflation_df` are correlated let's try a merge:

First, let's move the date column out of the index in `inflation_df`:

In [ ]:
inflation_df.reset_index(inplace=True, drop=False)
inflation_df.head()

In this case, `reset_index` will preserve the original index (`date`) as a new column because of `drop=False`. 

In [ ]:
try:
    df.merge(inflation_df, left_on="year_start", right_on="date")
except Exception as e:
    print(e)

This merge failed because the data types do not match up. `year_start` is an integer and `date` is an object. Let's change that:

In [ ]:
# df['date'] = pd.to_datetime(df['year_start'])
df['date'] = df['year_start'].apply(lambda x: datetime.datetime.strptime(f"{x}-12-31", "%Y-%m-%d"))
df.head()

Above I used `f"{x}-12-31"` to match how the `inflation_df` represents the date for the year (as opposed to the start of the year `f"{x}-01-01"`).

In [ ]:
inflation_df['date'] = inflation_df['date'].apply(lambda x: datetime.datetime.strptime(f"{x}", "%Y-%m-%d"))
inflation_df.head()

In [ ]:
print(inflation_df['date'].dtype, df['date'].dtype)

Now, `inflation_df['date']` and `df['date']` have the same data type, we can use `merge` on it:

In [ ]:
merged_df = df.merge(inflation_df, left_on="date", right_on='date')
merged_df.head()

A merge is a fast way to enrich our data based on corresponding values in two dataframes. The reason we do this is simple:

In [ ]:
merged_df['adj_salary_audit'] = merged_df['salary'] * merged_df['multiplier']
merged_df.head()

In [ ]:
merged_df['audit_delta'] = merged_df['adj_salary_audit'] - merged_df['adj_salary']

In [ ]:
total_adjusted = merged_df['adj_salary'].sum()
total_adjusted_usd = utils.float_to_dollars(total_adjusted)
total_adjusted_audit = merged_df['adj_salary_audit'].sum()
total_adjusted_audit_usd = utils.float_to_dollars(total_adjusted_audit)

In [ ]:
audit_delta_sum = utils.float_to_dollars(merged_df['audit_delta'].sum())

In [ ]:
difference_perc = ((total_adjusted_audit - total_adjusted) / total_adjusted_audit) * 100
print(f"Difference between our internal audit and their numbers is {difference_perc:.4f}% which totals to {audit_delta_sum}")

In [ ]:
difference_perc = ((total_adjusted_audit - total_adjusted) / total_adjusted_audit) * 100

In [ ]:
print("Total Adjusted Salary (usd)", total_adjusted_usd)
print("Total Adjusted Salary Audit (usd)", total_adjusted_audit_usd)

print("Delta Total", audit_delta_sum)
print(f"Detla Percent Difference {difference_perc:.4f}%")

This shows us that our adjusted salary number is about $410 million higher but that's under 1% difference. 

Since this data is good enough for future pandas lessons,  we're not going to dig any deeper in improving the adjusted salaries. But there's a few questions that come to mind on how we could do it:

- With this data, we used `year_start` and not `year_end` for our inflation rate multiplier. Perhaps `year_end` would yield closer results.
- The source datasets might *both* be incorrect; how would we change this?
- Does over `$410 million+` skew future results given total sum is over `$68 billion+`?

In [ ]:
# Export to samples dir

# merged_df.to_csv('samples/5-player-adj-salaries-audit.csv', index=False)